In [2]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt

In [23]:
# find chunks for each person
path_to_member_data = "../../../../originaldata/20111129_DI_Member.txt"
data = pd.read_csv(path_to_member_data, sep="\t")
def find_chunk(x):
    """
    Find the start and stop indices of the longest chunk of 12-month enrollments.
    
    Parameters
    ----------
    x : list of integers
    
    Returns
    -------
    (start, end) : tuple of nonnegative ints
        These give the start and end indices of x such that x[i] is equal
        to 12 for start <= i <= end. Gives the largest such chunk.
        If no such chunk exists, just returns (0,0).
    """
    chunks = {}
    chunk = []
    count = 0
    max_count = 0
    in_chunk = False
    for i in xrange(len(x)):
        if x[i]==12:
            in_chunk = True
            chunk.append(i)
            count += 1
        elif in_chunk:
            in_chunk = False
            max_count = max(max_count, count)
            chunks[count] = chunk
            chunk = []
            count = 0
    if in_chunk:
        max_count = max(max_count, count)
        chunks[count] = chunk
    try:
        return (chunks[max_count][0],chunks[max_count][-1])
    except KeyError:
        return (0,0)
data["Chunk_Indices"] = data.apply(lambda x: find_chunk(x[4:13]),axis=1)
data["Chunk_Start"] = data.apply(lambda x: x.loc["Chunk_Indices"][0], axis=1)
data["Chunk_End"] = data.apply(lambda x: x.loc["Chunk_Indices"][1], axis=1)
data["Chunk_Length"] = data["Chunk_End"]-data["Chunk_Start"]+1
del data["Chunk_Indices"]
data.to_csv("member_with_chunks.txt",index=False)

In [2]:
data = pd.read_csv("member_with_chunks.txt")
data["ID"] = data.iloc[:,0]
data = data.drop(data.columns[0],axis=1)
joined = pd.read_csv("../joined_data.csv")
merged = pd.merge(data, joined, left_on="ID", right_on="ID")
merged = merged.drop(merged.columns[[0,2,3,4,5,6,7,8,9,10,11,12,17,]],axis=1)

In [2]:
# helper functions to identify codes of interest

# CKD
ckd_filter_string = "substr({}, 1, 3) in ('403', '404', '582', '583', '585', '586') or {} = '5880'"
def checkCKD(c):
    return c[:3] in ('403', '404', '582', '583', '585', '586') or c == '5880'

# Type II Diabetes
diab_filter_string = "substr({}, 1, 3) = '250' and length(ICD) = 5 and substr({}, 5, 5) in ('0', '2')"
def checkDM(c):
    return len(c)==5 and c[:3] =='250' and c[4] in ('0', '2')

# Macular Degeneration
def checkMD(c):
    return c[:4] == '3625'

# glaucoma
def checkGC(c):
    return c[:3]=='365'

# diabetic retinopathy
def checkDR(c):
    # return c[:4] in ('3620','2505')
    return c[:4]=='3620'

In [5]:
merged = pd.read_csv("joined_data.csv", index_col=0)

In [18]:
# only keep people born in 1972 or earlier (so they would be at least 40 by 2012)
merged = merged[merged.Birth.apply(lambda x: x[:4]<='1972')]

In [19]:
merged.head()

,ID,Birth,Sex,Codes,Dates
156218,1878022,1972-12-31 00:00:00,F,"6202,5920,V2511,5929,6202,6259,5990,4619,5920,...","2012-11-26,2011-11-30,2012-05-30,2013-02-07,20..."
156219,540138702,1972-12-31 00:00:00,F,"V2512,5920,7936,5920,5990,5921,V1301,78791,591...","2011-09-16,2011-03-28,2011-07-28,2011-03-08,20..."
156220,540961543,1972-12-31 00:00:00,F,"30000,79389,78052,462,4619,V700,7862,61189,296...","2014-07-30,2014-03-13,2014-10-01,2012-01-20,20..."
156221,542553881,1972-12-31 00:00:00,M,"V5721,2572,1869,4019,2364,1869,1869,78052,7805...","2011-11-21,2010-07-28,2011-12-16,2010-08-09,20..."
156222,545107124,1972-12-31 00:00:00,F,"7061,38840,7804,V720,V720,9953,2382,7061,7061","2014-09-25,2014-12-16,2014-12-16,2013-01-28,20..."


In [24]:
for check, fname in zip([checkCKD,checkDM],["ckd_survival.csv","diabetes_survival.csv"]):
    day = np.timedelta64(1,'D')
    year = np.timedelta64(365,'D')
    obs_years = 2 # number of years to use for observation
    merged["ObsCodes"] = "" # codes seen during observed period
    merged["Age"] = 0 # age relative to end of observed period
    merged["Survival"] = -1 # time in days to survival
    merged["Event"] = 0 # 0 means censored, 1 means occurred
    count=0
    for i in merged.index:
        if count % 1000 == 0:
            print (count)
        count += 1
        # find the date of first diagnosis, use to get surival time
        codes = np.array(merged.loc[i,"Codes"].split(","))
        dates = np.array(merged.loc[i,"Dates"].split(","))
        sorted_dates = np.argsort(dates)
        codes = codes[sorted_dates]
        dates = dates[sorted_dates]
        ind = -1
        for j,c in enumerate(codes):
            if check(c):
                ind = j
                break
        if ind==-1: #no disease code was seen, so censored. survival time is easy
            merged.loc[i,"Survival"] = 3*365 # they survived the whole 3 years
        else: # a disease code was seen. find survival time
            merged.loc[i,"Event"] = 1
            merged.loc[i,"Survival"] = (np.datetime64(dates[ind]) - np.datetime64("2012-01-01"))/day+1
        # get observed codes and age for those with valid survival times
        if merged.loc[i,"Survival"] > 0:
                merged.loc[i,"ObsCodes"] = ",".join([c for k,c in enumerate(codes) if 
                                             dates[k][:4]<'2012'])
                merged.loc[i,"Age"] = (np.datetime64("2012-01-01") - 
                                       np.datetime64(merged.loc[i,"Birth"][:10]))/year
    merged[["ID","Sex","Age","ObsCodes","Survival","Event"]].to_csv(fname,index=False)        

0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000
30000
31000
32000
33000
34000
35000
36000
37000
38000
39000
40000
41000
42000
43000
44000
45000
46000
47000
48000
49000
50000
51000
52000
53000
54000
55000
56000
57000
58000
59000
60000
61000
62000
63000
64000
65000
66000
67000
68000
0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000
30000
31000
32000
33000
34000
35000
36000
37000
38000
39000
40000
41000
42000
43000
44000
45000
46000
47000
48000
49000
50000
51000
52000
53000
54000
55000
56000
57000
58000
59000
60000
61000
62000
63000
64000
65000
66000
67000
68000


In [3]:
diab = pd.read_csv("diabetes_survival.csv")

In [6]:
diab.fillna(value="").head()

,ID,Sex,Age,ObsCodes,Survival,Event
0,1878022,F,39.027397,"7061,7061,69010,1100,V7231,7061,V700,5990,5997...",1095.0,0
1,540138702,F,39.027397,"4611,5990,4659,7862,07999,78904,78904,5920,592...",1095.0,0
2,540961543,F,39.027397,"38870,4660,462",1095.0,0
3,542553881,M,39.027397,"2572,4011,78052,78052,2572,4019,78052,2572,401...",1095.0,0
4,545107124,F,39.027397,,1095.0,0


In [9]:
diab[(diab.Event==0)&(diab.Survival>0)].shape

(58745, 6)

In [108]:
[i for i,c in enumerate(joined.loc[984,"Codes"].split(",")) if c[:3] in ('403', '404', '582', '583', '585', '586')]

[139, 146, 152, 160, 166, 172]

In [ ]:
joined.loc[984,"Codes"].split(",").index()

In [109]:
joined.loc[984,"Dates"].split(",")[139]

'2009-05-13'